<a href="https://colab.research.google.com/github/gabriellaaileen/pillar-stability-prediction/blob/main/Pillar_Stability_Prediction_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Requirements**

In [1]:
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow import nn

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
import tensorflow as tf

In [3]:
!wget -P data/ https://raw.githubusercontent.com/gabriellaaileen/pillar-stability-prediction/main/"Data gabungan 2.csv"

--2022-08-30 14:02:10--  https://raw.githubusercontent.com/gabriellaaileen/pillar-stability-prediction/main/Data%20gabungan%202.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10806 (11K) [text/plain]
Saving to: ‘data/Data gabungan 2.csv’

Data gabungan 2.csv 100%[===================>]  10.55K  --.-KB/s    in 0s      

2022-08-30 14:02:11 (109 MB/s) - ‘data/Data gabungan 2.csv’ saved [10806/10806]



In [4]:
df = pd.read_csv('/content/data/Data gabungan 2.csv')

#**Exploratory Data Analysis**

In [ ]:
df.head(5)

,Depth,PW,BW,MH,Stability
0,76.2,4.88,6.10,1.37,Failed
1,29.0,5.40,6.30,2.90,Failed
2,60.0,7.00,6.00,1.82,Failed
3,53.0,5.60,6.10,1.80,Failed
4,21.0,6.75,5.25,3.20,Failed


In [ ]:
df.shape

(423, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Depth      423 non-null    float64
 1   PW         423 non-null    float64
 2   BW         423 non-null    float64
 3   MH         423 non-null    float64
 4   Stability  423 non-null    object 
dtypes: float64(4), object(1)
memory usage: 16.6+ KB


In [ ]:
#Menghitung presentase missing values dari masing-masing variabel
(df.isnull().sum()/len(df)*100).to_frame('Persentase Missing Value Dataset (%)')

,Persentase Missing Value Dataset (%)
Depth,0.0
PW,0.0
BW,0.0
MH,0.0
Stability,0.0


In [ ]:
df.describe()

,Depth,PW,BW,MH
count,423.000000,423.000000,423.000000,423.000000
mean,77.546194,9.948558,6.088818,3.000686
std,44.716598,4.919084,0.616702,1.122797
min,13.220000,2.740000,3.690000,1.000000
25%,45.335000,6.195000,5.975000,2.200000
50%,66.000000,9.000000,6.000000,2.920000
75%,96.000000,12.095000,6.375000,3.650000
max,254.450000,35.000000,10.000000,6.450000


In [ ]:
print(df.groupby(['Stability']).agg({"Stability": "count"}), end="\n\n\n")

           Stability
Stability           
Failed            86
Intact           337




In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.offline as py

fig = px.scatter(df, x="Depth", y="PW", color="Stability", size="BW")
fig

#**Preprocessing**

##Imbalance dataset

In [5]:
X, y = df.drop('Stability', axis=1),df['Stability']

###Oversampling

In [6]:
# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [7]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [8]:
X_resampled, y_resampled = ros.fit_resample(X, y)

###Hasil

In [9]:
print(X_resampled)

     Depth     PW    BW    MH
0     76.2   4.88  6.10  1.37
1     29.0   5.40  6.30  2.90
2     60.0   7.00  6.00  1.82
3     53.0   5.60  6.10  1.80
4     21.0   6.75  5.25  3.20
..     ...    ...   ...   ...
669  112.0  10.55  6.45  2.82
670   61.0   6.10  6.10  4.57
671   62.0   7.50  6.40  4.00
672   53.3   5.18  6.40  3.66
673   41.0   6.40  6.40  6.20

[674 rows x 4 columns]


In [10]:
print(y_resampled)

0      Failed
1      Failed
2      Failed
3      Failed
4      Failed
        ...  
669    Failed
670    Failed
671    Failed
672    Failed
673    Failed
Name: Stability, Length: 674, dtype: object


In [11]:
df=pd.concat([X_resampled, y_resampled], axis=1)
df.head()

,Depth,PW,BW,MH,Stability
0,76.2,4.88,6.10,1.37,Failed
1,29.0,5.40,6.30,2.90,Failed
2,60.0,7.00,6.00,1.82,Failed
3,53.0,5.60,6.10,1.80,Failed
4,21.0,6.75,5.25,3.20,Failed


In [12]:
print(df.groupby(['Stability']).agg({"Stability": "count"}), end="\n\n\n")

           Stability
Stability           
Failed           337
Intact           337




##Label Encoding

In [91]:
#Split dataset menjadi feature (X) dan label (y)
y = df['Stability']
X = df.drop(columns=['Stability'])

In [92]:
le = LabelEncoder()
y = le.fit_transform(y)

In [93]:
dictionary={}
for i in range(len(list(le.classes_))):
  dictionary[list(le.classes_)[i]]=i
dictionary

{'Failed': 0, 'Intact': 1}

#**Split Dataset (Train - Val - Test)**

##Train - Test Split

In [16]:
#Split train dan test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

##Train - Val - Test Split

In [17]:
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [18]:
X_train.shape

(539, 4)

In [19]:
X_valid.shape

(67, 4)

In [20]:
X_test.shape

(68, 4)

#**Ensemble Learning**

In [23]:
adam = Adam(learning_rate=0.001)

In [24]:
#Callback
callback = EarlyStopping(
    monitor="accuracy",
    patience=10,
    restore_best_weights=True,
    mode='auto'
)

##Ensemble Learning (ReLU, ELU, GELU)

In [31]:
model_1 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_2 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_3 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_1.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_2.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_3.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

history_1 = model_1.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_2 = model_2.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_3 = model_3.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)

Epoch 1/100
34/34 [==============================] - 2s 3ms/step - loss: 0.8895 - accuracy: 0.5584
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6965 - accuracy: 0.5770
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.6327
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6430 - accuracy: 0.6586
Epoch 5/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5854 - accuracy: 0.6531
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5790 - accuracy: 0.7106
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5461 - accuracy: 0.7087
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5539 - accuracy: 0.7217
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5673 - accuracy: 0.7310
Epoch 10/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5162 - accuracy: 0.7384
Epoch 11/

In [32]:
predictions_1=model_1.predict(X_test, batch_size=None)
predictions_2=model_2.predict(X_test, batch_size=None)
predictions_3=model_3.predict(X_test, batch_size=None)

In [33]:
predictions=['predictions_1','predictions_2','predictions_3']

In [36]:
label_1=[]
for i in range(len(predictions_1)):
  if predictions_1[i][0]>0.5:
    label_1.append(1)
  else:
    label_1.append(0)

In [37]:
label_2=[]
for i in range(len(predictions_2)):
  if predictions_2[i][0]>0.5:
    label_2.append(1)
  else:
    label_2.append(0)

In [38]:
label_3=[]
for i in range(len(predictions_3)):
  if predictions_3[i][0]>0.5:
    label_3.append(1)
  else:
    label_3.append(0)

In [39]:
testing = pd.DataFrame()
testing['Stability'] = y_test
testing['Pred_Stability_1'] = label_1
testing['Pred_Stability_2'] = label_2
testing['Pred_Stability_3'] = label_3

In [40]:
testing.head(10)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3
0,0,0,0,0
1,1,1,1,1
2,1,0,0,0
3,0,0,0,0
4,1,1,1,1
5,0,0,0,0
6,0,1,1,1
7,1,1,1,1
8,1,1,1,1
9,1,0,0,1


In [ ]:
testing['final_prediction'] = testing.mode(axis=1)[0]
testing['final_prediction'] = testing['final_prediction'].astype("int64")

In [46]:
testing.tail(20)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3,final_prediction
48,0,0,0,0,0
49,0,0,0,1,0
50,0,0,0,0,0
51,0,1,0,1,0
52,1,1,1,1,1
53,0,0,0,0,0
54,0,0,0,0,0
55,0,1,1,1,1
56,1,1,1,1,1
57,1,1,1,1,1


In [47]:
print(classification_report(testing['Stability'].values, testing['final_prediction'].values, zero_division=1))

              precision    recall  f1-score   support

           0       0.82      0.89      0.86        37
           1       0.86      0.77      0.81        31

    accuracy                           0.84        68
   macro avg       0.84      0.83      0.84        68
weighted avg       0.84      0.84      0.84        68



##Ensemble Learning (3 Linear)

In [79]:
model_1 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_2 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_3 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.linear),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_1.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_2.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_3.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

history_1 = model_1.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_2 = model_2.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_3 = model_3.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)

Epoch 1/100
34/34 [==============================] - 1s 4ms/step - loss: 31.8104 - accuracy: 0.5028
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.8033 - accuracy: 0.5492
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5903 - accuracy: 0.6809
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5453 - accuracy: 0.6753
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5323 - accuracy: 0.7254
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5484 - accuracy: 0.7273
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5197 - accuracy: 0.7310
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5632 - accuracy: 0.7050
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5452 - accuracy: 0.7069
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5163 - accuracy: 0.7310
Epoch 11

In [80]:
predictions_1=model_1.predict(X_test, batch_size=None)
predictions_2=model_2.predict(X_test, batch_size=None)
predictions_3=model_3.predict(X_test, batch_size=None)

In [81]:
predictions=['predictions_1','predictions_2','predictions_3']

In [82]:
label_1=[]
for i in range(len(predictions_1)):
  if predictions_1[i][0]>0.5:
    label_1.append(1)
  else:
    label_1.append(0)

In [83]:
label_2=[]
for i in range(len(predictions_2)):
  if predictions_2[i][0]>0.5:
    label_2.append(1)
  else:
    label_2.append(0)

In [84]:
label_3=[]
for i in range(len(predictions_3)):
  if predictions_3[i][0]>0.5:
    label_3.append(1)
  else:
    label_3.append(0)

In [85]:
testing = pd.DataFrame()
testing['Stability'] = y_test
testing['Pred_Stability_1'] = label_1
testing['Pred_Stability_2'] = label_2
testing['Pred_Stability_3'] = label_3

In [86]:
testing.head(10)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3
0,0,0,0,0
1,1,1,1,1
2,1,1,1,1
3,0,0,0,0
4,1,1,1,1
5,0,0,0,0
6,0,1,1,1
7,1,1,1,1
8,1,1,1,1
9,1,0,1,1


In [87]:
testing['final_prediction'] = testing.mode(axis=1)[0]
testing['final_prediction'] = testing['final_prediction'].astype("int64")

In [88]:
testing.tail(20)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3,final_prediction
48,0,0,0,1,0
49,0,0,1,1,0
50,0,0,0,0,0
51,0,1,1,1,1
52,1,1,1,1,1
53,0,0,0,0,0
54,0,0,0,0,0
55,0,0,1,1,0
56,1,1,1,1,1
57,1,1,1,1,1


In [89]:
print(classification_report(testing['Stability'].values, testing['final_prediction'].values, zero_division=1))

              precision    recall  f1-score   support

           0       0.91      0.86      0.89        37
           1       0.85      0.90      0.88        31

    accuracy                           0.88        68
   macro avg       0.88      0.88      0.88        68
weighted avg       0.88      0.88      0.88        68



##Ensemble Learning (3 ReLU)

In [58]:
model_1 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_2 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_3 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_1.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_2.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_3.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

history_1 = model_1.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_2 = model_2.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_3 = model_3.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)

Epoch 1/100
34/34 [==============================] - 1s 3ms/step - loss: 1.4042 - accuracy: 0.5009
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6624 - accuracy: 0.6160
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6646 - accuracy: 0.6197
Epoch 4/100
34/34 [==============================] - 0s 4ms/step - loss: 0.6387 - accuracy: 0.6549
Epoch 5/100
34/34 [==============================] - 0s 4ms/step - loss: 0.6051 - accuracy: 0.6772
Epoch 6/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5268 - accuracy: 0.7143
Epoch 7/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5217 - accuracy: 0.7310
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5168 - accuracy: 0.7421
Epoch 9/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5057 - accuracy: 0.7440
Epoch 10/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5293 - accuracy: 0.7291
Epoch 11/

In [59]:
predictions_1=model_1.predict(X_test, batch_size=None)
predictions_2=model_2.predict(X_test, batch_size=None)
predictions_3=model_3.predict(X_test, batch_size=None)

In [60]:
predictions=['predictions_1','predictions_2','predictions_3']

In [61]:
label_1=[]
for i in range(len(predictions_1)):
  if predictions_1[i][0]>0.5:
    label_1.append(1)
  else:
    label_1.append(0)

In [62]:
label_2=[]
for i in range(len(predictions_2)):
  if predictions_2[i][0]>0.5:
    label_2.append(1)
  else:
    label_2.append(0)

In [63]:
label_3=[]
for i in range(len(predictions_3)):
  if predictions_3[i][0]>0.5:
    label_3.append(1)
  else:
    label_3.append(0)

In [64]:
testing = pd.DataFrame()
testing['Stability'] = y_test
testing['Pred_Stability_1'] = label_1
testing['Pred_Stability_2'] = label_2
testing['Pred_Stability_3'] = label_3

In [66]:
testing['final_prediction'] = testing.mode(axis=1)[0]
testing['final_prediction'] = testing['final_prediction'].astype("int64")

In [67]:
testing.tail(20)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3,final_prediction
48,0,0,1,0,0
49,0,0,0,0,0
50,0,0,0,0,0
51,0,0,1,0,0
52,1,1,1,1,1
53,0,0,0,0,0
54,0,0,0,0,0
55,0,1,1,1,1
56,1,1,1,1,1
57,1,1,1,1,1


In [68]:
print(classification_report(testing['Stability'].values, testing['final_prediction'].values, zero_division=1))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88        37
           1       0.87      0.84      0.85        31

    accuracy                           0.87        68
   macro avg       0.87      0.87      0.87        68
weighted avg       0.87      0.87      0.87        68



##Ensemble Learning (3 ELU)

In [69]:
model_1 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_2 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_3 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.elu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_1.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_2.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_3.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

history_1 = model_1.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_2 = model_2.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_3 = model_3.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)

Epoch 1/100
34/34 [==============================] - 1s 3ms/step - loss: 1.5160 - accuracy: 0.5325
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.7452 - accuracy: 0.5158
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.7760 - accuracy: 0.4917
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.7042 - accuracy: 0.5046
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6697 - accuracy: 0.5807
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6185 - accuracy: 0.6419
Epoch 7/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5678 - accuracy: 0.6753
Epoch 8/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5139 - accuracy: 0.7570
Epoch 9/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5192 - accuracy: 0.7254
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5090 - accuracy: 0.7310
Epoch 11/

In [70]:
predictions_1=model_1.predict(X_test, batch_size=None)
predictions_2=model_2.predict(X_test, batch_size=None)
predictions_3=model_3.predict(X_test, batch_size=None)

In [71]:
predictions=['predictions_1','predictions_2','predictions_3']

In [72]:
label_1=[]
for i in range(len(predictions_1)):
  if predictions_1[i][0]>0.5:
    label_1.append(1)
  else:
    label_1.append(0)

In [73]:
label_2=[]
for i in range(len(predictions_2)):
  if predictions_2[i][0]>0.5:
    label_2.append(1)
  else:
    label_2.append(0)

In [74]:
label_3=[]
for i in range(len(predictions_3)):
  if predictions_3[i][0]>0.5:
    label_3.append(1)
  else:
    label_3.append(0)

In [75]:
testing = pd.DataFrame()
testing['Stability'] = y_test
testing['Pred_Stability_1'] = label_1
testing['Pred_Stability_2'] = label_2
testing['Pred_Stability_3'] = label_3

In [76]:
testing['final_prediction'] = testing.mode(axis=1)[0]
testing['final_prediction'] = testing['final_prediction'].astype("int64")

In [77]:
testing.tail(10)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3,final_prediction
58,0,0,0,0,0
59,1,1,1,1,1
60,1,1,1,1,1
61,1,1,1,1,1
62,0,0,0,0,0
63,1,1,1,1,1
64,0,0,0,0,0
65,1,1,1,1,1
66,0,1,1,1,1
67,1,0,1,1,1


In [78]:
print(classification_report(testing['Stability'].values, testing['final_prediction'].values, zero_division=1))

              precision    recall  f1-score   support

           0       0.89      0.86      0.88        37
           1       0.84      0.87      0.86        31

    accuracy                           0.87        68
   macro avg       0.87      0.87      0.87        68
weighted avg       0.87      0.87      0.87        68



##Ensemble Learning (3 GELU)

In [48]:
model_1 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_2 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_3 = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(512, input_shape=[4],activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(256, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(128, activation=tf.keras.activations.gelu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
                                    ])
model_1.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_2.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
model_3.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])

history_1 = model_1.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_2 = model_2.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)
history_3 = model_3.fit(X_train, y_train,batch_size=16,epochs=100,verbose=1,callbacks=callback)

Epoch 1/100
34/34 [==============================] - 1s 4ms/step - loss: 1.4159 - accuracy: 0.4842
Epoch 2/100
34/34 [==============================] - 0s 4ms/step - loss: 0.7570 - accuracy: 0.5325
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.6643 - accuracy: 0.6011
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5873 - accuracy: 0.6790
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5801 - accuracy: 0.6976
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.5687 - accuracy: 0.7143
Epoch 7/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5406 - accuracy: 0.7254
Epoch 8/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5309 - accuracy: 0.7421
Epoch 9/100
34/34 [==============================] - 0s 4ms/step - loss: 0.4966 - accuracy: 0.7551
Epoch 10/100
34/34 [==============================] - 0s 5ms/step - loss: 0.4629 - accuracy: 0.7774
Epoch 11/

In [49]:
predictions_1=model_1.predict(X_test, batch_size=None)
predictions_2=model_2.predict(X_test, batch_size=None)
predictions_3=model_3.predict(X_test, batch_size=None)

In [50]:
label_1=[]
for i in range(len(predictions_1)):
  if predictions_1[i][0]>0.5:
    label_1.append(1)
  else:
    label_1.append(0)

In [51]:
label_2=[]
for i in range(len(predictions_2)):
  if predictions_2[i][0]>0.5:
    label_2.append(1)
  else:
    label_2.append(0)

In [52]:
label_3=[]
for i in range(len(predictions_3)):
  if predictions_3[i][0]>0.5:
    label_3.append(1)
  else:
    label_3.append(0)

In [53]:
testing = pd.DataFrame()
testing['Stability'] = y_test
testing['Pred_Stability_1'] = label_1
testing['Pred_Stability_2'] = label_2
testing['Pred_Stability_3'] = label_3

In [54]:
testing.head(10)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3
0,0,0,0,0
1,1,1,1,1
2,1,1,0,0
3,0,0,0,0
4,1,1,1,1
5,0,0,0,0
6,0,0,0,1
7,1,1,1,1
8,1,1,1,1
9,1,1,1,0


In [55]:
testing['final_prediction'] = testing.mode(axis=1)[0]
testing['final_prediction'] = testing['final_prediction'].astype("int64")

In [56]:
testing.tail(20)

,Stability,Pred_Stability_1,Pred_Stability_2,Pred_Stability_3,final_prediction
48,0,0,1,0,0
49,0,1,1,0,0
50,0,0,0,0,0
51,0,0,0,1,0
52,1,0,1,1,1
53,0,0,0,0,0
54,0,0,0,0,0
55,0,0,1,1,0
56,1,1,1,1,1
57,1,1,1,1,1


In [57]:
print(classification_report(testing['Stability'].values, testing['final_prediction'].values, zero_division=1))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95        37
           1       0.97      0.90      0.93        31

    accuracy                           0.94        68
   macro avg       0.94      0.94      0.94        68
weighted avg       0.94      0.94      0.94        68

